In [67]:
import pandas as pd 
import requests
from io import StringIO

response = requests.get('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20201120&type=ALLBUT0999&_=1606035295283')
response.text[:1000]

'"109年11月20日 價格指數(臺灣證券交易所)"\r\n"指數","收盤指數","漲跌(+/-)","漲跌點數","漲跌百分比(%)","特殊處理註記",\r\n"寶島股價指數","15,653.04","+","1.24","0.01","",\r\n"發行量加權股價指數","13,716.44","-","5.99","-0.04","",\r\n"臺灣公司治理100指數","8,124.27","-","9.25","-0.11","",\r\n"臺灣50指數","11,138.51","-","22.14","-0.20","",\r\n"臺灣50權重上限30%指數","10,480.90","-","12.44","-0.12","",\r\n"臺灣中型100指數","9,823.47","+","31.24","0.32","",\r\n"臺灣資訊科技指數","18,995.06","-","8.14","-0.04","",\r\n"臺灣發達指數","8,576.00","-","6.37","-0.07","",\r\n"臺灣高股息指數","6,741.84","+","6.58","0.10","",\r\n"臺灣就業99指數","7,314.05","+","0.83","0.01","",\r\n"臺灣高薪100指數","6,871.37","-","0.28","--","",\r\n"未含金融指數","11,974.78","+","0.69","0.01","",\r\n"未含電子指數","15,087.42","-","15.00","-0.10","",\r\n"未含金融電子指數","12,874.04","+","5.88","0.05","",\r\n"小型股300指數","7,354.82","+","21.45","0.29","",\r\n"水泥窯製類指數","630.58","+","0.39","0.06","",\r\n"塑膠化工類指數","822.35","+","3.38","0.41","",\r\n"機電類指數","3,693.21","-","0.30","-0.01","",\r\n"水泥類指數","171.11","-","0.64","-0.37","",\r\n"食品類指數","1,741.01

In [68]:
lines = response.text.split('\n')
lines[300]

'="00733","富邦臺灣中小","223,800","18","5,525,199","24.49","24.70","24.49","24.70","+","0.25","24.64","10","24.70","1","0.00",\r'

In [69]:
newlines = []

for line in lines:
    if len(line.split('",')) == 17:
        newlines.append(line)

# dataframe

In [70]:
df = pd.read_csv(StringIO('\n'.join(newlines).replace('=',"")))
df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,0050,元大台灣50,"4,800,675","3,751","546,485,967",114.15,114.15,113.35,114.00,-,0.15,114.00,80,114.05,3,0.00,NaN
1,0051,元大中型100,"65,245",42,"2,633,376",40.39,40.42,40.24,40.38,X,0.00,40.32,1,40.45,4,0.00,NaN
2,0052,富邦科技,"1,144,797",340,"115,563,910",100.65,101.05,100.10,100.90,-,0.15,100.90,8,100.95,1,0.00,NaN
3,0053,元大電子,"33,294",17,"1,768,840",53.10,53.25,53.05,53.25,X,0.00,53.00,1,53.35,12,0.00,NaN
4,0054,元大台商50,"16,107",7,"422,854",26.41,26.41,26.16,26.16,X,0.00,26.10,54,26.20,1,0.00,NaN


# string

In [71]:
df = df.astype(str)

In [72]:
#def func(s):
    #return s.str.replace(',','')
#df = df.apply(func)

df = df.apply(lambda  s: s.str.replace(',',''))

df = df.set_index('證券代號')
df.head()

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
證券代號,,,,,,,,,,,,,,,,
0050,元大台灣50,4800675,3751,546485967,114.15,114.15,113.35,114.00,-,0.15,114.00,80,114.05,3,0.00,nan
0051,元大中型100,65245,42,2633376,40.39,40.42,40.24,40.38,X,0.0,40.32,1,40.45,4,0.00,nan
0052,富邦科技,1144797,340,115563910,100.65,101.05,100.10,100.90,-,0.15,100.90,8,100.95,1,0.00,nan
0053,元大電子,33294,17,1768840,53.10,53.25,53.05,53.25,X,0.0,53.00,1,53.35,12,0.00,nan
0054,元大台商50,16107,7,422854,26.41,26.41,26.16,26.16,X,0.0,26.10,54,26.20,1,0.00,nan


In [78]:
df = df.apply(lambda s: pd.to_numeric(s,errors='coerce'))#coerce是指如果今天轉為數值失敗的話直接轉為NaN
#df.isnull().sum() == len(df)#選取columns裡面全部都是NaN的
df = df[df.columns[df.isnull().sum() != len(df)]] 
df.head()

,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
0050,4800675,3751,546485967,114.15,114.15,113.35,114.00,0.15,114.00,80,114.05,3,0.0
0051,65245,42,2633376,40.39,40.42,40.24,40.38,0.00,40.32,1,40.45,4,0.0
0052,1144797,340,115563910,100.65,101.05,100.10,100.90,0.15,100.90,8,100.95,1,0.0
0053,33294,17,1768840,53.10,53.25,53.05,53.25,0.00,53.00,1,53.35,12,0.0
0054,16107,7,422854,26.41,26.41,26.16,26.16,0.00,26.10,54,26.20,1,0.0


In [82]:
df[df['收盤價']/df['開盤價']>1.05] #收盤價大於開盤價 5%的股票


,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
1307,1680527,997,38026344,21.40,23.20,21.40,22.70,1.35,22.70,10,22.75,19,42.83
2007,6448605,884,47345887,6.80,7.37,6.80,7.37,0.67,7.37,1234,NaN,0,0.00
2033,493117,176,5328184,9.99,10.90,9.99,10.90,0.99,10.90,243,NaN,0,19.12
2106,6614185,3142,233998674,34.10,36.25,34.10,36.00,1.90,35.95,21,36.00,224,48.65
2429,576125,242,11706449,18.95,20.40,18.95,20.40,1.85,20.40,110,NaN,0,0.00
2443,2107894,335,6521437,3.06,3.41,3.06,3.30,0.10,3.30,7,3.31,2,0.00
2465,9227404,4206,189883805,20.30,22.00,19.60,21.45,0.35,21.45,166,21.50,21,0.00
2613,1503989,564,27547212,17.95,19.00,17.70,19.00,1.05,18.90,30,19.00,34,100.00
3006,38222961,12646,1749096627,43.35,46.65,43.30,46.65,4.20,46.65,9724,NaN,0,12.18
